In [64]:
import gym
import numpy as np
from scipy.spatial.distance import cdist


# Define the environment
class TSPEnv(gym.Env):
  def __init__(self, n_cities=100, show_debug_data = False):
    self.n_cities = n_cities
    self.xy = (np.random.rand(self.n_cities,2)*100).round(2)
    self.x=self.xy[:,0]
    self.y=self.xy[:,1]
    self.step_counter = 0
    self.show_debug_data = show_debug_data

    #print(f'genrated stops xy: {self.xy}')
    self.distance_matrix = cdist(self.xy,self.xy,'euclidean').round(0)

    # self.distance_matrix = np.array([[0, 2448, 1434, 1260, 2045],
    #                   [2448, 0, 2546, 959, 2367],
    #                   [1434, 2546, 0, 2408, 1745],
    #                   [1260, 959, 2408, 0, 2295],
    #                   [2045, 2367, 1745, 2295, 0]])
    self.current_city = np.random.randint(n_cities)
    self.visited_cities = [self.current_city]
    self.remaining_cities = [i for i in range(n_cities) if i != self.current_city]
    # Define the action space
    self.action_space = gym.spaces.Discrete(n_cities)

    # Define the observation space
    self.observation_space = gym.spaces.Box(low=0, high=1, shape=(n_cities,), dtype=np.float32)

    print(f'Start in Init: {self.current_city}')

  def reset(self):
    self.step_counter = 0
    self.current_city = np.random.randint(self.n_cities)
    self.visited_cities = [self.current_city]
    self.remaining_cities = [i for i in range(self.n_cities) if i != self.current_city]
    return self._get_observation()

  def step(self, action):
    self.step_counter += 1
    if action < len(self.remaining_cities):
      next_city = self.remaining_cities[action]
      reward = -self.distance_matrix[self.current_city][next_city]
      self.remaining_cities.remove(next_city)
      self.visited_cities.append(next_city)
      self.current_city = next_city
      done = len(self.remaining_cities) == 0
      
      if(self.show_debug_data):
        print(f'Action in step: {action}')
        print(f'Reward in step: {reward}')
        print(f'Current city in step: {self.current_city}')
        print(f'Remaining city in step: {self.remaining_cities}')
        print(f'Visited city in step: {self.visited_cities}')
        print(f'Stepcounter in step: {self.step_counter}')
      return self._get_observation(), reward, done, {}
    else:
      return self._get_observation(), 0, False, {}


  def _get_observation(self):
    observation = np.zeros(self.n_cities)
    observation[self.current_city] = 1
    return observation

  
  def _test_distance(self,CurrentCity, NextCity):
    return -self.distance_matrix[CurrentCity][NextCity]
    




In [ ]:
env = TSPEnv()

env._test_distance(1,3)

env.step(0)

env.step(1)
env.step(2)
env.step(3)

In [99]:
import random
import numpy as np
from collections import deque

import torch
import torch.nn as nn
import torch.optim as optim

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the DQN model
class DQN(nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.fc1 = nn.Linear(input_dim, 512)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(512, 256)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(256, output_dim)

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu1(x)
    x = self.fc2(x)
    x = self.relu2(x)
    x = self.fc3(x)
    return x


# Define the DQNAgent
class DQNAgent:
  def __init__(self, env, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, 
               alpha=1e-3, alpha_decay=0.01, gamma=0.99, memory_size=10000, 
               batch_size=64):
    self.env = env
    self.epsilon = epsilon
    self.epsilon_decay = epsilon_decay
    self.epsilon_min = epsilon_min
    self.alpha = alpha
    self.alpha_decay = alpha_decay
    self.gamma = gamma
    self.memory = deque(maxlen=memory_size)
    self.batch_size = batch_size

    # Define the model and the target model
    self.model = DQN(env.observation_space.shape[0], env.action_space.n).to(device)
    self.target_model = DQN(env.observation_space.shape[0], env.action_space.n).to(device)
    self.target_model.load_state_dict(self.model.state_dict())

    # Define the optimizer
    self.optimizer = optim.Adam(self.model.parameters(), lr=alpha)

    # Define the loss function
    self.loss_fn = nn.MSELoss()

  def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

  def act(self, state):
    if np.random.rand() <= self.epsilon:
      return self.env.action_space.sample()
    else:
      state = torch.from_numpy(state).float().to(device)
      q_values = self.model(state)
      return q_values.argmax().item()

  def update(self):
    # Don't update if there are not enough samples in the memory
    if len(self.memory) < self.batch_size:
      return

    # Sample a batch from the memory
    samples = random.sample(self.memory, self.batch_size)

    # Split the batch into separate variables
    states, actions, rewards, next_states, dones = zip(*samples)

    # Convert variables to tensors and move them to the device
    states = torch.from_numpy(np.vstack(states)).float().to(device)
    actions = torch.from_numpy(np.vstack(actions)).long().to(device)
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(device)
    dones = torch.from_numpy(np.vstack(dones)).float().to(device)

    # Calculate the Q values for the current states
    q_values = self.model(states)
    q_values = q_values.gather(1, actions)

    # Calculate the Q values for the next states
    next_q_values = self.target_model(next_states).max(1)[0].unsqueeze(1)

    # Calculate the target Q values
    target_q_values = rewards + (self.gamma * next_q_values * (1 - dones))

    # Calculate the loss
    loss = self.loss_fn(q_values, target_q_values)

    # Perform backpropagation
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    # Update the target model
    for target_param, param in zip(self.target_model.parameters(), self.model.parameters()):
      target_param.data.copy_(param.data * (1 - self.alpha_decay) + target_param.data * self.alpha_decay)

    # Update the epsilon value
    self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)
  
  def SaveAgent(self):
    torch.save(self.model.state_dict(),'model.pt')
    print('Model saved')



In [100]:
import gym
import numpy as np


# Instantiate the environment
env = TSPEnv(50)

# Instantiate the agent
agent = DQNAgent(env)

# Set the number of episodes to run
n_episodes = 1

#lists for learning evaluation
scores = []


# Run the episodes
for episode in range(n_episodes):
  # Reset the environment and get the initial state
  state = env.reset()

  # Set the initial reward to 0
  total_reward = 0
  total_steps = 0

  while True:
    # Take an action
    action = agent.act(state)

    # Step the environment
    next_state, reward, done, _ = env.step(action)

    # Remember the experience
    agent.remember(state, action, reward, next_state, done)

    # Update the state and the reward
    state = next_state
    total_reward += reward
    total_steps += 1

    # Update the agent
    agent.update()

    # If the episode is done, break the loop
    if done:
      break
  
  scores.append((episode, total_reward, total_steps))
  # Print the total reward for the episode
  print(f"Episode: {episode+1}, Reward: {total_reward}, Steps needed: {total_steps}")

agent.SaveAgent()


Start in Init: 49
Episode: 1, Reward: -2364.0, Steps needed: 348


In [ ]:
print(scores)

[('Episode', 0, -2369.0, 185), ('Episode', 1, -2595.0, 22739), ('Episode', 2, -2337.0, 15193), ('Episode', 3, -2550.0, 9175), ('Episode', 4, -2330.0, 11747), ('Episode', 5, -2863.0, 9510), ('Episode', 6, -2555.0, 8727), ('Episode', 7, -2586.0, 13904), ('Episode', 8, -2451.0, 8929), ('Episode', 9, -2707.0, 17729)]
